In [18]:

from Bio.Blast import NCBIXML
from Bio import SeqIO
import subprocess

"""
批量在NCBI与SWISSPRO下载同源蛋白
"""

"""
同源蛋白去冗余
"""

"""
tblastn生成XML文件
"""

"""
提取XML文件hsp.sbjct氨基酸序列，生成sbjct_protein文件
根据XML文件的hit_id提取基因组hit_macth区上下游1500bp的DNA，生成sbjct_genome文件
"""
handle = open('/home/guisen/IL/IL-17-cal.xml','rt') #加载XML文件
blast_results = NCBIXML.parse(handle)#解析
i = 0#自增变量初始值
list_num = []#自增变量列表收集
for blast_record in blast_results:
    for alignment in blast_record.alignments:
        hsp_stat = []
        hsp_end = []
        for hsp in alignment.hsps:
            hsp_stat.append(hsp.sbjct_start)
            hsp_end.append(hsp.sbjct_end)
        print(f'{hsp_stat}\t{hsp_end}')
        print(blast_record.query)
        for seq_record in SeqIO.parse('/home/guisen/03.structure_annotation/genome.fa','fasta'):
            if alignment.hit_id.split('|')[0].strip() == seq_record.id.strip():
                print(min(hsp_stat))
                print(max(hsp_end))
                if min(hsp_stat) - 1500 < 0:
                    seq_start = 0
                else:
                    seq_start = min(hsp_stat) - 1500
                if max(hsp_end) + 1500 < len(seq_record.seq):
                        seq_end = max(hsp_stat) + 1500
                else:
                    seq_end = len(seq_record.seq)
                i += 1
                list_num.append(i)
                #sbjct_protein = open(f'/home/guisen/tmp/sbjct_protein{i}.fa','w+')
                #sbjct_protein.write(f'>query_id:{blast_record.query}|alignment_id:{alignment.title}|strat:{hsp.sbjct_start}|end:{hsp.sbjct_end}|e_value:{hsp.expect}|score:{hsp.score}\n{hsp.sbjct}')
                #sbjct_protein.close()
                sbjct_genome = open(f'/home/guisen/tmp/sbjct_genome{i}.fa','w+')
                sbjct_genome.write(f'>{seq_record.id}\n{seq_record.seq[seq_start:seq_end]}')
                sbjct_genome.close()
                print(f'提取第{i}个Hit成功')#显示
            else:
                pass
        for seq_protein in SeqIO.parse('/home/guisen/IL/IL-17-cal.fa','fasta'):
                if blast_record.query == seq_protein.id:
                    sbjct_protein = open(f'/home/guisen/tmp/sbjct_protein{i}.fa','w+')
                    sbjct_protein.write(f'>query_id:{blast_record.query}\n{seq_protein.seq}')
                    sbjct_protein.close()
                                       
"""
并行化genewise，批量预测基因
"""
for i in list_num:
    cline = subprocess.Popen(f'genewise sbjct_protein{i}.fa sbjct_genome{i}.fa -both -gff -cdna -pep -genes>> sbjct_predicted.gff',env={'PATH':'/opt/biosoftware/wise2.4.1/src/bin','WISECONFIGDIR':'/opt/biosoftware/wise2.4.1/wisecfg/'},shell=True,cwd='/home/guisen/tmp/')
print('completed')

"""

"""

[38689495, 38690967, 38689173]	[38689620, 38691080, 38689337]
evm.model.Lachesis13__1343704387.1154
38689173
38691080
提取第1个Hit成功
[35120180, 35122182]	[35120509, 35122385]
evm.model.Lachesis12__1644025036.1116
35120180
35122385
提取第2个Hit成功
[40339545]	[40339772]
evm.model.Lachesis12__1644025036.1116
40339545
40339772
提取第3个Hit成功
[9039068]	[9039298]
evm.model.Lachesis2__1753060648.194
9039068
9039298
提取第4个Hit成功
[9064962, 9064636]	[9065204, 9064794]
evm.model.Lachesis2__1753060648.195
9064636
9065204
提取第5个Hit成功
[49360120, 49363020, 49363376]	[49360356, 49363283, 49363438]
evm.model.Lachesis2__1753060648.1294
49360120
49363438
提取第6个Hit成功
[35120285]	[35120509]
evm.model.Lachesis2__1753060648.1294
35120285
35120509
提取第7个Hit成功
[40337043, 40339548]	[40337321, 40339796]
evm.model.Lachesis5__547510254.1454
40337043
40339796
提取第8个Hit成功
[35120282]	[35120485]
evm.model.Lachesis5__547510254.1454
35120282
35120485
提取第9个Hit成功
completed


'\n\n'